In [1]:
# https://witestlab.poly.edu/blog/capture-and-decode-fm-radio/
# 2021-07-20
# miniconda Python 3.9.5
#
# inlining svg image in a markdown cell
# 0. ![](livefm-system-diagram.svg "livefm-system-diagram.svg")
# 1. Using markdown syntax: ![Alt text](https://mirrors.creativecommons.org/presskit/logos/cc.logo.svg)
# 2. Using HTML: <img style="float: right;" src="https://mirrors.creativecommons.org/presskit/logos/cc.logo.svg">
#

In [2]:
from rtlsdr import RtlSdr  
import numpy as np  
import scipy.signal as signal
import scipy.io.wavfile
import matplotlib.pyplot as plt
#
sdr = RtlSdr()

In [3]:
# In the next command, we will specify the frequency at which to capture samples. 
# We will use 107.3MHz, Rutgers Radio:
F_station = int(107.3e6)   # 漢聲廣播電台  
F_offset  = 250000         # Offset to capture at  
#
# We capture at an offset to avoid DC spike
Fc = F_station - F_offset  # Capture center frequency  
Fs = int(1140000)          # Sample rate  
# N  = int(8192000)        # Samples to capture, 7 seconds long 
# N  = int(16384000)       # Samples to capture,14 seconds long  
N  = int(16384000*2)       # Samples to capture,28 seconds long  

# configure device
sdr.sample_rate = Fs      # Hz  
sdr.center_freq = Fc      # Hz  
sdr.freq_correction = 30  # PPM  <----- test
sdr.gain = 'auto'

# Read samples
samples = sdr.read_samples(N)

# Clean up the SDR device
sdr.close()  
del(sdr)
#

In [4]:
# Convert samples to a numpy array
x1 = np.array(samples).astype("complex64")
fc1 = np.exp(-1.0j*2.0*np.pi*F_offset/Fs*np.arange(len(x1)))  
x2 = x1 * fc1 
#
f_bw = 200000  
dec_rate = int(Fs / f_bw)  
x4 = signal.decimate(x2, dec_rate)  
#
y5 = x4[1:] * np.conj(x4[:-1])  
x5 = np.angle(y5)
#
# Given a signal 'x5' (in a numpy array) with sampling rate Fs_y
Fs_y = Fs/dec_rate
d = Fs_y * 75e-6   # Calculate the # of samples to hit the -3dB point  
x = np.exp(-1/d)   # Calculate the decay between each sample  
b = [1-x]          # Create the filter coefficients  
a = [1,-x]  
x6 = signal.lfilter(b,a,x5) 
#
# Now we can decimate once again to focus on the mono audio part of the broadcast:
# Find a decimation rate to achieve audio sampling rate between 44-48 kHz
audio_freq = 44100.0  
dec_audio = int(Fs_y/audio_freq)  
Fs_audio = Fs_y / dec_audio
x7 = signal.decimate(x6, dec_audio) 
#
# finally, we can write to an audio file:
# Scale audio to adjust volume
x7 *= 10000 / np.max(np.abs(x7))  
# Save to file as 16-bit signed single-channel audio samples
x7.astype("int16").tofile("./FM-103.7MZ-mono-2.raw") 
#
scipy.io.wavfile.write("./FM-103.7MZ-mono-2.wav", 44100, x7.astype("int16"))
#
print('Done')

Done
